# Fine-tuning to predict the driver gene on example dataset

## Training

### Follow the README.md for training preparations, including datasets and pretrained files. 

In [1]:
## Here we use the toy dataset 'set3_example_train.h5ad' and 'set3_example_test.h5ad' as an example. 
## The dataset link has been given in README.md.

import os

os.system('bash launch_train.sh')

## Testing 

### Load results and evaluate. 

In [2]:
## Here we load results on step 1000 as an example. 
## The link for 'checkpoint-step-1000.pth' has been given in README.md.

import os

os.system('python load_results.py -c 1000')

In [3]:
import pandas as pd
import scanpy as sc
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


df = pd.read_csv('set3_test_example_results.csv', index_col=0)
df['pred_gene'] = df.idxmax(axis=1)
print(df.head())


                      ABCB10    AKAP12      ALX4  APOBEC3C  APOBEC3D  \
TGCATCCTCGATCCAA-4 -7.821463 -1.156066  0.904565 -5.127326 -7.951521   
TTGGATGGTATCCTCC-2 -7.477587 -1.947112 -1.975892 -1.696544 -7.003691   
AACCACAGTCTCCCTA-1 -9.641579 -3.978698 -5.091012 -5.724802 -5.395859   
TAGAGTCTCATGGATC-4 -1.639565 -2.339934  0.846601 -3.660203 -7.996193   
TTTCAGTTCCATTCGC-2 -3.726980 -3.369183 -4.184911  0.105774 -1.985862   

                       APOL2   ARHGDIB    BICDL2      CBY1       CD2  ...  \
TGCATCCTCGATCCAA-4 -1.957271  2.360333 -5.758391 -6.885813 -9.281010  ...   
TTGGATGGTATCCTCC-2 -2.546962  1.258157 -2.950146 -0.769823 -4.229147  ...   
AACCACAGTCTCCCTA-1 -5.997128 -5.460759 -0.255089 -5.033724 -3.466392  ...   
TAGAGTCTCATGGATC-4 -2.124491  1.050665 -0.941778  1.620585 -2.813236  ...   
TTTCAGTTCCATTCGC-2 -2.452409  2.454847 -1.646882 -0.831029 -1.429325  ...   

                       TBX21  TNFRSF1A   TNFRSF1B   TNFRSF9  TNFRSF12A  \
TGCATCCTCGATCCAA-4 -5.753242 -

In [4]:
adata = sc.read_h5ad('../dataset_full/set3_example_test.h5ad')
perturb_gene = adata.obs['perturbation'].values


perturb_gene = adata.obs['perturbation'].values
pred_gene = df['pred_gene'].values


accuracy = accuracy_score(perturb_gene, pred_gene)
f1 = f1_score(perturb_gene, pred_gene, average='weighted')


print(f"Testing accuracy (for toy dataset): {accuracy:.3f}")
print(f"Testing F1 score (for toy dataset): {f1:.3f}")

Testing accuracy (for toy dataset): 0.510
Testing F1 score (for toy dataset): 0.498
